# Transform In Transformer 介绍

**TNT(Transform In Transformer)** 是继ViT 和 DeiT之后的优异vision transformer(视觉transformer)。在视觉任务上有较好性能。

整体上来说，TNT较以前的模型在transformer处理上有更多的细节上的提升。

提升点:

  1. **patch-level + pixel-level** 两级结合，即利用Attention特性对patch-level上图像的**全局特征**进行高质量提取，同时又利用pixel-level对全局下的**局部特征**进行进一步提取，保证了图片的较为完整空间关系。

  2. **position encode** 进行位置编码，保证图片在split时空间结构被较好地保留下来，这是以前的视觉transformer不具有的。

  3. pixel-level的embedding实现嵌入后还将继续嵌入到patch-level的embedding中。

  4. embedding的结果会与position encode进行结合，保证图片特征提取过程中**完整的空间结构**。
   
严谨性：
	
  1. 利用消融实验，对**head数，position encode，two-level**的必要性进行了实验证明。
  
  
> 下面就TNT复现代码进行讲解，适当的补充TNT体系结构的说明。

## 完整实现代码

	-- models
		
        * tnt_layers.py:  TNT模型所涉及到的组件网络层实现 + 代码注释
        
        * tnt_model.py:  TNT模型实现、与基本small、big模型的配置 + 代码注释
        
        * tset.py:     提供TNT模型的简单使用方法
        
        
> 下边是一个测试文件，其它信息可前往test.py中查看

In [10]:
%cd models/
from test import create_tnt_by_basecfg

# on_start_test: True, 表示进行基本测试
model = create_tnt_by_basecfg(num_classes = 10, img_size = 224, in_chans = 3, choice_big=False, on_start_test=True)

/home/aistudio/models
(1, 10)


# 一、TNT 模型基本流程解析

(图源:论文)

![](https://ai-studio-static-online.cdn.bcebos.com/7bb8e1ce274841918d0bea9e29efc5b00ff343594f76455bbbed1227a32968ee)


基本流程如图中所示，基本步骤总结如下：


| 步骤 | 组件 | info |
| -------- | -------- | -------- |
| 第一步     | Unfold + Conv2D     |   将输入图片分割成指定的pixel大小和patch大小   |
| 第二步     | TNT Block     | 堆叠的TNT块将patch与pixel作为输入进行处理     |
| 第三步     | inner transformer     | 在TNT中处理pixel     |
| 第四步     | outer transformer     | 在TNT中处理patch    |
| 第五步     | TNT Blocks     | 反复第二到第四步，直到第L块运行结束    |
| 第六步     | MLP head     | 将TNT Blocks输出中的class_token部分作为输入，从而得到分类任务的输出结果    |


# 二、TNT代码解析

就上边所说的流程进行代码复现，为了描述完整性，按照以下顺序介绍：

1. Attention部分

2. MLP部分

3. DropPath部分(添加的丢弃策略)

4. TNT Block部分

5. Pixel Embed部分

以上顺序与表格中略有不同，增添部分为论文代码实现的一些策略。

> 具体内容，将在代码中去介绍，每一个部分的代码前边，会有对该部分代码的主要解析，与参数说明

## 0. 基本的依赖库

In [1]:
import paddle                              # 提供数据操作方法
from paddle import nn                      # 网络层API
from paddle.nn import functional as F      # 常见方法

import math
import os

## 1. Attention部分

注意力部分，主要是通过对输入数据进行注意力编码，然后将注意力结果叠加到原输入数据上。

Attention前后数据形状不发生改变，仅发生数据值变化。

主要实现思路:

1. 将输入进行指定维度的线性映射(Linear)

2. 第一次映射，同时获取questions 和 keys的映射数据

3. 然后分离两者，并将两者进行矩阵乘积，然后通过softmax，计算question在key上的注意力结果

4. 第二次映射，获取values的映射数据

5. question与key的注意力结果与value进行矩阵乘积，将注意力作用到value上

6. 最后将value映射回输入大小，适当丢弃，最后输出


> 下边是代码实现 + 代码注释

In [2]:
class Attention(nn.Layer):
    '''
        Multi-Head Attention
    '''
    def __init__(self, in_dims, hidden_dims, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0., logging=False):
        super(Attention, self).__init__()
        ''' Attention
            params list:
                in_dims:    输入维度大小
                hidden_dim: 隐藏层维度大小
                num_heads:  注意力头数量
                qkv_bias:   是否对question、keys、values开启映射
                attn_drop:  注意力丢弃率
                proj_drop:  映射丢弃率
                logging:    是否输出Attention的init参数日志
        '''

        # 确保输入层、隐藏层维度为偶数，且不为零，否则在头划分映射时会发生大小错误
        assert in_dims % 2 == 0 and in_dims != 0, \
            'please make sure the input_dims(now: {0}) is an even number.(%2==0 and !=0)'.format(in_dims)
        assert hidden_dims % 2 == 0 and hidden_dims != 0, \
            'please make sure the hidden_dims(now: {0}) is an even number.(%2==0 and !=0)'.format(hidden_dims)

        self.in_dims     = in_dims                           # ATT输入大小
        self.hidden_dims = hidden_dims                       # ATT隐藏层大小
        self.num_heads   = num_heads                         # ATT的头数目
        self.head_dims   = hidden_dims // num_heads          # 将ATT隐藏层大小按照头数平分，作为ATT-head的维度大小
        self.scale       = self.head_dims ** -0.5            # 缩放比例按照头的唯独大小进行开(-0.5次幂)
        self.qkv_bias    = qkv_bias
        self.attn_drop   = attn_drop
        self.proj_drop   = proj_drop

        # 输出日志信息
        if logging:
            print('\n—— Attention Init-Logging ——')
            print('{0:20}'.format(list(dict(in_dims=self.in_dims).keys())[0]),         ': {0:12}'.format(self.in_dims))
            print('{0:20}'.format(list(dict(hidden_dims=self.hidden_dims).keys())[0]), ': {0:12}'.format(self.hidden_dims))
            print('{0:20}'.format(list(dict(num_heads=self.num_heads).keys())[0]),     ': {0:12}'.format(self.num_heads))
            print('{0:20}'.format(list(dict(head_dims=self.head_dims).keys())[0]),     ': {0:12}'.format(self.head_dims))
            print('{0:20}'.format(list(dict(scale=self.scale).keys())[0]),             ': {0:12}'.format(self.scale))
            print('{0:20}'.format(list(dict(qkv_bias=self.qkv_bias).keys())[0]),       ': {0:12}'.format(self.qkv_bias))
            print('{0:20}'.format(list(dict(attn_drop=self.attn_drop).keys())[0]),     ': {0:12}'.format(self.attn_drop))
            print('{0:20}'.format(list(dict(proj_drop=self.proj_drop).keys())[0]),     ': {0:12}'.format(self.proj_drop))

        '''
            questions + keys  |  values
                    project layers
        '''
        # questions + keys 的映射层: *2 就是在一次操作下将两层一同映射
        # qkv_bias：是否开启bias --> 开启默认全零初始化
        self.qk = nn.Linear(self.in_dims, self.hidden_dims*2, bias_attr=qkv_bias)
        # values 的映射层
        # qkv_bias：是否开启bias --> 开启默认全零初始化
        self.v  = nn.Linear(self.in_dims, self.in_dims, bias_attr=qkv_bias)
        # ATT的丢弃层
        self.attn_drop = nn.Dropout(attn_drop)

        '''
            注意力结果映射层
        '''
        self.proj = nn.Linear(self.in_dims, self.in_dims)
        self.proj_drop = nn.Dropout(proj_drop)
    

    @paddle.jit.to_static
    def forward(self, inputs):
        x = inputs
        B, N, C= x.shape           # B:batch_size, N:patch_number, C:input_channel

        # print('\n—— Attention Forward-Logging ——')
        # print('{0:20}'.format(list(dict(B=B).keys())[0]),                      ': {0:12}'.format(B))
        # print('{0:20}'.format(list(dict(N=N).keys())[0]),                      ': {0:12}'.format(N))
        # print('{0:20}'.format(list(dict(C=C).keys())[0]),                      ': {0:12}'.format(C))

        # 利用输入映射question 和 keys维度的特征
        # print('input(x): ', x.numpy().shape)
        qk = self.qk(x)          # 将输入映射到question + keys上
        # print('qk_project: ', qk.numpy().shape)
        qk = paddle.reshape(qk, shape=(B, N, 2, self.num_heads, self.head_dims))   # 将question + keys分离
        # print('qk_reshape: ', qk.numpy().shape)
        qk = paddle.transpose(qk, perm=[2, 0, 3, 1, 4])   # 重新排列question和keys的数据排布
        # print('qk_transpose: ', qk.numpy().shape)
        '''
            ①上面实现的划分，正好对应: head_dims = hidden_dims // num_heads
            ②排布更新为: 映射类别(question+keys)，batch_size, head_number, patch_number, head_dims
        '''
        q, k = qk[0], qk[1]          # 分离question 和 keys
        # print('q: ', q.numpy().shape)
        # print('k: ', k.numpy().shape)

        # 利用输入映射 values 维度的特征
        v = self.v(x).reshape(shape=(B, N, self.num_heads, -1)).transpose(perm=(0, 2, 1, 3))
        # print('v: ', v.numpy().shape)

        # 通过question 与 keys矩阵积，计算patch的注意力结果
        attn = paddle.matmul(q, k.transpose(perm=(0, 1, 3, 2))) * self.scale
        # print('attn_matrix*: ', attn.numpy().shape)
        '''
            k.transpose(perm=(0, 1, 3, 2)) : 最后两维发生转置 --> 用于矩阵乘法，实现注意力大小计算(question 对 keys)
            * self.scale : 针对注意力头数进行一定的缩放，稳定值
        '''
        attn = F.softmax(attn, axis=-1)          # 通过softmax整体估算注意力 -- 对每一个patch上的hidden_dim进行注意力计算
        # print('attn_softmax: ', attn.numpy().shape)
        attn = self.attn_drop(attn)              # 丢弃部分注意力结果

        # 将注意力结果与value进行矩阵乘法结合
        x = paddle.matmul(attn, v).transpose(perm=(0, 2, 1, 3)).reshape(shape=(B, N, -1))
        # print('x_matrix*: ', x.numpy().shape)
        ''' 
            attn 与 v 矩阵乘: 实现注意力叠加
            transpose(perm=(0, 2, 1, 3)): 将patch与head维度互换(转置) -- 保证reshape不发生错误合并
            reshape(shape=(B, N, -1)): 转换回:batch_size, patch_num, out_dims形式 -- out_dims = num_head * head_dims
        '''
        x = self.proj(x)          # 将注意力叠加完成的结果进行再映射，将其映射回输入大小
        # print('x_proj: ', x.numpy().shape)
        x = self.proj_drop(x)     # 丢弃部分结果
        return x

## 2. MLP部分

多层感知机部分是整个过程中最简单的部分，只需要两层线性层(Linear)即可

主要实现思路:

1. 确定输入、输出、隐藏层大小

2. 构建一层输入层，实现输入大小到隐藏大小的映射

3. 构建一层输出层，实现隐藏大小到输出大小的映射

4. 构建前向时，输入层后先跟GELU激活函数，再跟丢弃层

5. 然后经过输出层 + 丢弃层，得到MLP的输出

> 下边是代码实现 + 代码注释

In [ ]:
class MLP(nn.Layer):
    '''
        两层fc的感知机
    '''
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super(MLP, self).__init__()
        ''' MLP
            params list:
                in_features:      输入大小
                hidden_features:  隐藏层大小
                out_features:     输出大小
                act_layer:        激活层
                drop:             丢弃率
        '''

        # 如果前项为None，则返回后向作为赋值内容
        out_features    = out_features    or in_features
        hidden_features = hidden_features or in_features
        
        # 第一层输入
        self.fc1  = nn.Linear(in_features,     hidden_features)
        self.act  = act_layer()
        # 第二层输出
        self.fc2  = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout()
    

    @paddle.jit.to_static
    def forward(self, inputs):
        x = inputs
        
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)

        return x

## 3. DropPath部分

路径丢弃，与普通的Dropout相比，路径丢弃使得丢弃的数据更多。

所谓路径，就是沿着shape中的第一个维度进行一整个的丢弃(全部置为0)；

而普通的丢弃，仅仅对全部参数进行随机的丢弃，而DropPath则是将实现从逐个元素跨越到了整个轴上。

> drop_path: 考虑到大面积丢弃，所以对未丢弃的数据的值进行了一定的扩增

主要实现思路:

1. 首先在运行前判断是否进行丢弃 —— drop为0，不丢弃；在训练中，不丢弃

2. 计算保持率

3. 得到路径丢弃的随机丢弃分布 -- random_tensor

4. 扩增数据，并实现丢弃


> 下边是代码实现 + 代码注释

In [ ]:
class DropPath(nn.Layer):
    '''删除路径数据
        延一个路径进行丢弃(沿数据第一个维度进行丢弃)
        丢弃的对应path下，所有数据置为0
    '''
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        ''' DropPath
            params list:
                drop_prob:      丢弃率
        '''
        self.drop_prob = drop_prob


    @paddle.jit.to_static
    def forward(self, inputs):
        x = inputs
        return self.drop_path(x)  # self.training是否在训练模型下


    def drop_path(self, x):
        '''
            具体的path丢弃操作: 改变对应的值，不改变数据形状
        '''
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = paddle.to_tensor([1 - self.drop_prob])
        # 作batch_size维度大小的shape结构--(batch_size, 1, 1, ...)
        shape = (x.shape[0], ) + (1, ) * (x.ndim - 1)                        # batch_size, 1*原ndim减去batch_size维度的大小
        random_tensor = keep_prob + paddle.rand(shape=shape, dtype=x.dtype)  # 按照划分的shape创建一个[0, 1)均匀分布随机tensor
        # 利用[0,1)均匀分布产生的值 + 保持率，就可以实现等比例的保留和丢弃
        # 由于随机性，可以保证丢弃的随机性
        # 由于值总是在[0,1)间，所以只要得到的值 + keep_prob大于一个阈值，就保留
        # 但是因为值时均匀分布的，虽然每一个位置上值时随机取到的，但是确实均匀划分的，
        # 因此这样相加后可以实现对应丢弃概率下的丢弃path，并非一定会执行丢弃
        # print(keep_prob + paddle.rand(shape=[2,]))
        # print(paddle.floor(keep_prob + paddle.rand(shape=[2,])))
        random_tensor = paddle.floor(random_tensor)           # 将1作为阈值，从而floor向下取整筛选满足的数据
        # 仅仅留下 0, 1
        # print(random_tensor)

        # print(x[0, 0, 0])
        # print(keep_prob)
        # print(paddle.divide(x, keep_prob)[0, 0, 0])

        # print(random_tensor.shape)
        # print('x: ', x.numpy())
        output = paddle.divide(x, keep_prob) * random_tensor
        # print('output: ', output.numpy())
        return output

## 4. TNT Block部分

TNT Block是TNT模型中的特征提取结构。

TNT Block由Tin 与 Tout组成，利用Tin 对 pixel进行特征提取；利用Tout 对 patch进行特征提取。

具体流程：先Tin，然后再Tout

主要实现思路:

1. 先构建Tin，利用Attention + MLP + Linear实现

2. 输入的pixel_embed嵌入数据先通过Attention提取，作为当前层的pixel_embed结果，

3. 再通过MLP进行映射提取，又进行叠加，得到当前block最终的pixel数据

4. 然后利用Linear 对 pixel进行映射，实现从pixel到patch的映射，并将其加到patch_embed嵌入数据中

5. 再构建Tout, 只需Attention + MLP

6. 将上边加过pixel_embed的patch_embed依次通过Attention和MLP，得到Block的输出


> 下边是代码实现 + 代码注释

In [ ]:
class TNT_Block(nn.Layer):
    '''
        实现inner transfromer 和 outer transformer, 从pixel-level 和 patch-level进行数据特征提取

        特性：
            输入输出前后，tensor不发生shape变化（中间过程存在部分映射有shape变化）
    '''
    def __init__(self, patch_embeb_dim, in_dim, num_pixel, out_num_heads=12, 
                 in_num_head=4, mlp_ratio=4.,qkv_bias=False, drop=0., 
                 attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        '''TNT_Block
            params list:
                patch_embeb_dim     : patch的嵌入维度大小(也是实际输入数据的映射空间大小)
                in_dim              : 单个patch的维度大小(不包含pixel-level维度)
                num_pixel           : patch下的(in_dim维度)元素对应pixel的比例 1：num_pixel，即pixel个数（也是论文中指的patch2pixel分辨率）
                out_num_heads       : 输出(outer attn)的注意力头
                in_num_head         : 输入(inner attn)的注意力头
                mlp_ratio           : outer transformer中感知机隐藏层的维度缩放率
                qkv_bias            : question 、 keys 、values对应的(线性)映射层bias启用标记
                drop                : MLP部分、proj部分的丢弃率
                attn_drop           : attn部分的丢弃率
                drop_path           : 路径丢弃的丢弃率
                act_layer           : 激活层
                norm_layer          : 归一化层
        ''' 
        super(TNT_Block, self).__init__()

        # Inner transformer
        # 输入-注意力计算 -- pixel level
        self.in_attn_norm = norm_layer(in_dim)        # 层归一化--attention的归一化层
        self.in_attn = Attention(in_dims=in_dim, hidden_dims=in_dim,
                                 num_heads=in_num_head, qkv_bias=qkv_bias,
                                 attn_drop=attn_drop, proj_drop=drop)         # attention输出，tensor的shape不变
        # 输入-多层感知机进行维度映射
        self.in_mlp_norm = norm_layer(in_dim)        # 层归一化--mlp的归一化层
        self.in_mlp = MLP(in_features=in_dim, hidden_features=int(in_dim*4),
                          out_features=in_dim, act_layer=act_layer, drop=drop) # mlp输出，tensor的shape不变
        # 输入-线性映射输出
        self.in_proj_norm = norm_layer(in_dim)        # 层归一化--proj的归一化层
        self.in_proj = nn.Linear(in_dim * num_pixel, patch_embeb_dim, bias_attr=True)           # proj输出，tensor的shape发生改变


        # outer transformer
        # 输出-注意力计算 -- patch level
        self.out_attn_norm = norm_layer(patch_embeb_dim)
        self.out_attn = Attention(in_dims=patch_embeb_dim, hidden_dims=patch_embeb_dim,
                                  num_heads=out_num_heads, qkv_bias=qkv_bias,
                                  attn_drop=attn_drop, proj_drop=drop)
        
        self.out_mlp_norm = norm_layer(patch_embeb_dim)
        self.out_mlp = MLP(in_features=patch_embeb_dim, hidden_features=int(patch_embeb_dim * mlp_ratio),
                       out_features=patch_embeb_dim, act_layer=act_layer, drop=drop)

        # 公用方法
        # 路径丢弃
        self.drop_path = DropPath(drop_prob=drop_path) if drop_path > 0. else self.Identity  # self.Identity()占位方法，不对数据做任何处理


    @paddle.jit.to_static
    def forward(self, pixel_embeb, patch_embeb):
        '''
            params list:
                pixel_embeb: 上一个block输出的pixel-level out tensor
                patch_embeb: 上一个block输出的patch-level out tensor
        '''
        # inner work
        # 1. 注意力嵌入 added
        pixel_embeb = pixel_embeb + self.drop_path(self.in_attn(self.in_attn_norm(pixel_embeb)))
        # 2. mlp嵌入   added
        pixel_embeb = pixel_embeb + self.drop_path(self.in_mlp(self.in_mlp_norm(pixel_embeb)))
        # pixel嵌入的pathc叠加，在outer中完成

        # outer work
        B, N, C = patch_embeb.shape    # B:batch_size  N:Patch_Number  C:Feature_map_channel
        # 线性映射pixel到patch维度，N-1 means；映射前后不包括class_token
        # 映射是需要完整映射，不需要路径丢弃
        pixel_embeb_proj2patch = self.in_proj(self.in_proj_norm(pixel_embeb).reshape(shape=(B, N-1, -1)))
        # patch叠加上pixel的embeb数据，从patch1 --> patchn
        # 不在这里操作class_token
        patch_embeb[:, 1:] = patch_embeb[:, 1:] + pixel_embeb_proj2patch
        # 1. 注意力嵌入 added
        patch_embeb = patch_embeb + self.drop_path(self.out_attn(self.out_attn_norm(patch_embeb)))
        # print(patch_embeb.shape)
        # 2. mlp嵌入   added
        patch_embeb = patch_embeb + self.drop_path(self.out_mlp(self.out_mlp_norm(patch_embeb)))
        
        return pixel_embeb, patch_embeb


    def Identity(self, x):
        '''
            do nothing, only return input
        '''
        return x

## 5. Pixel Embed部分

Pixel Embed的像素嵌入部分是整个TNT模型的入口，实现对图像进行合理提取，根据预置的patch大小等参数生成pixel的嵌入数据。

【这里得到的pixel将可以在后边模型组网中实现从pixel2patch的直接映射——这就是TNT中提出的two-level extract(提取) features】

主要实现思路:

1. 根据给定的参数信息，进行一层卷积得到指定通道的特征图 -- 这是的通道数实际上就是单个pathc的维度（在不包含pixel级是的大小）【个人理解，如果有错误，可以在评论区指导一下，谢谢！】

2. 根据stride，进行卷积提取——实际上卷积核大小与padding刚好可以保证卷积前后图像大小不变，但由于stride，导致特征图缩小了

3. 再经过一个滑窗函数，进行滑窗展开，得到完整的pixel表示（pixel嵌入结果）

4. 输出pixel嵌入结果

> 下边是代码实现 + 代码注释

In [ ]:
class Pixel_Embed(nn.Layer):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, in_dim=48, stride=4):
        super(Pixel_Embed, self).__init__()
        '''Pixel_Embed: 像素嵌入--完成后才有patch嵌入
            params_list:
                img_size:   输入图片大小
                patch_size: 当前一个patch的预置大小
                in_chans:   输入的图像通道数
                in_dim:     设定的输入维度 -- 即预定的patch的个数
                stride:     分块时，使用卷积、滑窗的步长，决定着patch向下划分pixel时的分辨率(不是patch的分辨率)
        '''
        self.img_size = img_size
        self.num_patches = (img_size // patch_size) ** 2
        # 平方解释：img_size是宽时，//patch_size得到一行可划分多少个，而同样的列就有多少个
        # 这里考虑完整划分patch的个数
        self.in_dim = in_dim     # 每一个patch对应的分辨率 -- 即patch-level的分辨率
        self.new_patch_size = math.ceil(patch_size / stride)   # 向上取整 -- 确定向下划分pixel的分辨率
        self.stride = stride     # 卷积 + 滑窗的步长
        
        '''
            两步实现图像到patch的映射，与patch到pixel的分割
        '''
        self.proj = nn.Conv2D(in_channels=in_chans, out_channels=self.in_dim,
                              kernel_size=7, padding=3, stride=self.stride)
        # 7 // 2 == 3, padding = 3, conv后会保持原图大小 -- 在stride=1时
        self.unfold = F.unfold
        # 对输入提取滑动块
    

    @paddle.jit.to_static
    def forward(self, inputs, pixel_pos):
        x = inputs
        B, C, H, W = x.shape
        # 验证是否与所需大小一致
        assert H == self.img_size and W == self.img_size, \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size}*{self.img_size})."
            
        x = self.proj(x)
        x = self.unfold(x, kernel_sizes=self.new_patch_size, strides=self.stride)   # 提取滑块，获得对应的滑块结果
        # unfold将img转换为(B, Cout, Lout)
        # Cout = Channel * kernel_sizes[0] * kernel_sizes[1]  , 即每一次滑窗在图片上得到参数个数
        # Lout = hout * wout      —— 滑动block的个数
        # hout，wout 类似卷积在图片对应h，w上的滑动次数
        x = paddle.transpose(x, perm=[0, 2, 1])   # to be shape: (B, Lout, Cout)
        x = paddle.reshape(x, shape=(B * self.num_patches, self.in_dim, self.new_patch_size, self.new_patch_size))
        # 再分解为需要的编码形式: (Batch_size * patch_number, in_dim, new_patch_size, new_patch_size)
        # Batch_size * patch_number：将每个batch得到的patch数乘以batch_size得到总的patch数量
        # in_dim: 当前设定的输入维度大小
        # new_patch_size: 由stride确定的 patch 的分辨率 -- 对应patch下feature map的w，h
        x = x + pixel_pos           # 加上位置编码
        x = paddle.reshape(x, shape=(B * self.num_patches, self.in_dim, -1))  # 拼接pixel-level的元素
        x = paddle.transpose(x, perm=[0, 2, 1])
        # 转换为(B * self.num_patches, patch_dim2pixel_size, self.in_dim)
        # patch_dim2pixel_size: 即原in_dim下所有序列元素的拼接大小
        # 原来是，in_dim对应dim下的pixels
        # 现在是，每一个pixel对应in_dim的情况
        
        return x

# 三、TNT模型构建

(图源:论文)

![](https://ai-studio-static-online.cdn.bcebos.com/7bb8e1ce274841918d0bea9e29efc5b00ff343594f76455bbbed1227a32968ee)

接下来就上面实现的组件代码进行模型构建。

主要构建流程：

1. 首先构建 Pixel Embed， 并搭建从pixel到patch的映射网络， 完成图上第一步和第二步的操作

2. 创建class_token标记，为分类任务创建标记 -- 初始化采用随机截断正态分布

3. 创建pixel 与 patch的position encoder，实现论文中的位置编码 -- 初始化采用随机截断正态分布

4. 构建TNT Blocks，搭建特征提取主要网络 -- 完成图中第三和第四步

5. 构建head，对TNT Blocks的输出的class_token进行指定任务的输出 -- 实现分类结果的输出 -- 完成后几步

6. 预测结果并非限定在0-1.之间，实际使用预测，还需添加softmax函数进行预测

In [ ]:
class TNT(nn.Layer):

    def __init__(self, img_size=224, patch_size=16, in_chans=3, 
                 num_classes=10, embed_dim=768, in_dim=48, depth=12,
                 out_num_heads=12, in_num_head=4, mlp_ratio=4., qkv_bias=False, 
                 drop_rate=0., attn_drop_rate=0.,drop_path_rate=0., 
                 norm_layer=nn.LayerNorm, first_stride=4):
        super(TNT, self).__init__()
        '''TNT
            params_list:
                img_size:               输入图片大小（提前明确）
                patch_size:             patch的大小（提前明确）
                in_chans:               输入图片通道数
                num_classes:            分类类别
                embed_dim:              嵌入维度大小  -- 也是总的feature大小
                in_dim:                 每个patch的维度大小 -- 
                                        但不是每个patch对应的实际全部元素，
                                        全部元素还要加上in_dim[x]*每一个大小对应划分pixel的数量
                depth:                  深度(block数量)
                out_num_heads:          outer transformer的header数
                in_num_head:            inner transformer的header数
                mlp_ratio:              outer中mlp的隐藏层缩放比例
                qkv_bias:               question、keys、values是否启用bias
                drop_rate:              mlp、一般层（比如:映射输出、位置编码输出时）丢弃率
                attn_drop_rate:         注意力丢弃率
                drop_path_rate:         路径丢弃率
                norm_layer:             归一化层
                first_stride:           图片输入提取分块的步长--img --> patch
        '''

        self.num_classes = num_classes                      # 分类数
        self.embed_dim = embed_dim                          # 嵌入维度 == 特征数
        self.num_features = self.embed_dim                  # 嵌入维度 == 特征数

        # 先完成pixel-level的嵌入
        self.pixel_embeb = Pixel_Embed(img_size=img_size, patch_size=patch_size,
                                       in_chans=in_chans, in_dim=in_dim, stride=first_stride)
        self.num_patches = self.pixel_embeb.num_patches        # 当前pixel等效的实际的patch个数
        self.new_patch_size = self.pixel_embeb.new_patch_size  # 当前等效的每一个patch对应的pixel的分辨率(w == h)
        self.num_pixel = self.new_patch_size ** 2  # 当前每个patch实际划分的分辨率，w*h = w**2 得到patch2pixel的序列大小
        
        # 在进行patch-level嵌入
        # 从pixel映射到patch上，要对每一个patch展开为pixel下的数据通过层归一化
        self.first_proj_norm_start = norm_layer(self.num_pixel * in_dim)      # self.nwe_pixel * in_dim, 即每一个patch对应的全部元素
        # 然后映射到指定嵌入维度上
        self.first_proj = nn.Linear(self.num_pixel * in_dim, self.embed_dim)  # 将全部每一个patch对应的pixel都映射到指定嵌入维度大小的空间
        # 在经过一次归一化，输出
        self.first_proj_norm_end = norm_layer(self.embed_dim)

        # 分类标记
        # 截断正态分布来填充初始化cls_token、patch_pos、pixel_pos
        self.cls_token = paddle.create_parameter(shape=(1, 1, self.embed_dim), dtype='float32', attr=nn.initializer.TruncatedNormal(std=0.02))
        # 位置编码
        # patch_position_encode: self.num_patches + 1对应实际patch数目+上边的分类标记
        self.patch_pos = paddle.create_parameter(shape=(1, self.num_patches + 1, self.embed_dim), dtype='float32', attr=nn.initializer.TruncatedNormal(std=0.02))
        # pixel_position_encode: in_dim对应每一个patch的大小, self.new_patch_size对应patch划分为pixel的分辨率
        self.pixel_pos = paddle.create_parameter(shape=(1, in_dim, self.new_patch_size, self.new_patch_size), dtype='float32', attr=nn.initializer.TruncatedNormal(std=0.02))
        # 位置编码的丢弃
        self.pos_drop = nn.Dropout(p=drop_rate)

        # 在TNT中使用了path_drop, 进行路径丢弃
        # 为了丢弃更具随机性，提高鲁棒性，进行随机丢弃率的制作 -- 根据深度生成对应数量的丢弃率
        drop_path_random_rates = [r.numpy().item() for r in paddle.linspace(0, drop_path_rate, depth)]
        # 相同块，采用迭代生成
        tnt_blocks = []
        for i in range(depth):  # 根据深度添加TNT块
            tnt_blocks.append(
                TNT_Block(patch_embeb_dim=self.embed_dim, in_dim=in_dim, num_pixel=self.num_pixel,       # 嵌入大小， patch-level大小，patch2pixel大小
                          out_num_heads=out_num_heads, in_num_head=in_num_head, mlp_ratio=mlp_ratio,     # outer transformer头数，inner transformer头数，感知机隐藏层缩放比
                          qkv_bias=qkv_bias, drop=drop_rate, attn_drop=attn_drop_rate,                   # transormer中bias启动情况，映射层的丢弃率， 注意力层的丢弃率
                          drop_path=drop_path_random_rates[i], norm_layer=norm_layer)                    # 路径丢弃的丢弃率，归一化层
            )
        # 放入顺序层结构中
        self.tnt_blocks = nn.Sequential(*tnt_blocks)                # 输入前后不发生shape变化
        # tnt_blocks最后的输出还要经过一层归一化
        self.tnt_block_end_norm = norm_layer(self.embed_dim)             # 沿用前边最初归一化层的嵌入大小进行归一化设置

        # 输出任务结果 -- 这里是利用cls_token进行分类，embed_dim是整个模型的嵌入维度大小，也是cls_token的最后1维度的大小
        self.head = nn.Linear(self.embed_dim, self.num_classes) if self.num_classes > 0 else TNT_Block.Identity()
        
        # 初始化网络参数
        self._init_weights()


    def _init_weights(self):
        '''
            完成整个网络的初始化工作
        '''
        for l in self.sublayers():
            if isinstance(l, nn.Linear):
                # 随即截断正态分布填充初始化
                l.weight = paddle.create_parameter(shape = (l.weight.shape), dtype=l.weight.dtype,
                                                   name=l.weight.name, attr=nn.initializer.TruncatedNormal(std=0.02))
                # bias 默认开启就初始化为0  -- 不指定其它初始化方式时
            elif isinstance(l, nn.LayerNorm):
                # bias 默认开启就初始化为0  -- 不指定其它初始化方式时
                # 常量1.0初始化
                l.weight = paddle.create_parameter(shape = (l.weight.shape), dtype=l.weight.dtype,
                                                   name=l.weight.name, attr=nn.initializer.Constant(value=1.0))

    def get_classfier(self):
        '''
            用于获取分类任务头进行相应的任务预测、输出等
            [在当前任务不是必要的]
        '''
        return self.head


    def reset_classfier(self, num_classes):
        '''
            用于修改分类任务头的分类数目
        '''
        self.num_classes = num_classes  # 修改模型分类参数
        self.head = nn.Linear(self.embeb_dim, self.num_classes) if self.num_classes > 0 else TNT_Block.Identity()
        
    
    def upstream_forward(self, inputs):
        '''
            上有任务前向运算：TNT特征提取部分的网络运算
        '''
        x = inputs 
        B = x.shape[0]   # batch_size

        # 1. 先进行pixel-level的嵌入
        pixel_embeb = self.pixel_embeb(x, self.pixel_pos)

        # 2. 再从pixel-level上升到patch-level的嵌入(即向上映射)
        # 依次通过: pixel2patch的layer_norm, 然后进行映射，最后再通过一层layer_norm完成整个映射过程
        # 其中输入的pixel_embeb要经过shape变换，将散布在不同in_dim下的参数进行拼接到对应patch下
        patch_embeb = self.first_proj_norm_end(self.first_proj(self.first_proj_norm_start(pixel_embeb.reshape(shape=(B, self.num_patches, -1)))))
        patch_embeb = paddle.concat([self.cls_token, patch_embeb], axis=1)    # 将分类任务标记拼接到patch的嵌入空间中
        patch_embeb = patch_embeb + self.patch_pos   # 加上位置编码
        patch_embeb = self.pos_drop(patch_embeb)     # 丢弃一部分编码结果

        for tnt_block in self.tnt_blocks:            # 将前期处理好的嵌入信息，进行迭代，进行信息地进一步提取
            pixel_embeb, patch_embeb = tnt_block(pixel_embeb, patch_embeb)
        
        patch_embeb = self.tnt_block_end_norm(patch_embeb)
        
        return patch_embeb[:, 0]        # 0号位置为cls_token对应的位置

    @paddle.jit.to_static
    def forward(self, inputs):
        x = inputs
        
        # 主体前向传播
        x = self.upstream_forward(x)  # 返回cls_token，用于分类用
        # 分类任务
        x = self.head(x)  # 执行分类

        return x

# 四、简单总结

复现TNT整体上来说没那么多复杂的设计，但是其Conv+滑窗、position encoder、two-level的设计思想却有一定的吸收难度。

通过two-level，对全局特征和局部特征进行融合，改善以前的transformer在视觉上的不足。

利用position encoder巩固图像的空间结构。

参数调整建议:
	
    1.in_num_head 尽量不动；
    
    2.改embeding_size可以按照64的倍数增加；
    
    3.改动size，dim等参数均要被2整除才可保证模型运行
    
    4.mlp_ratio 在过拟合时，可以适当缩小
    
> 具体的代码都在models中，test.py为测试代码

> 有问题欢迎评论区讨论

> 姓名：蔡敬辉

> 学历：大三（在读）

> 爱好：喜欢参加一些大大小小的比赛，不限于计算机视觉——有共同爱好的小伙伴可以关注一下哦~后期会持续更新一些自制的竞赛baseline和一些竞赛经验分享

> 主要方向：目标检测、图像分割与图像识别--在学习NLP, 正在捣鼓FPGA

> 联系方式：qq:3020889729 微信:cjh3020889729

> 学校：西南科技大学